In [13]:
def remove_newlines(serie):
    serie = serie.str.replace('\n', ' ')
    serie = serie.str.replace('\\n', ' ')
    serie = serie.str.replace('  ', ' ')
    serie = serie.str.replace('  ', ' ')
    return serie

In [14]:
import os
import pandas as pd

# Create a list to store the text files
texts = []

# Get all the text files in the text directory
for file in os.listdir("text/"):
    # Open the file and read the text
    with open("text/" + file, "r") as f:
        text = f.read()

        texts.append((file, text))

# Create a dataframe from the list of texts
df = pd.DataFrame(texts, columns=['fname', 'text'])

# Set the text column to be the raw text with the newlines removed
df['text'] = remove_newlines(df.text)
df.to_csv('processed/scraped.csv')
df.head()

C:\Users\gaagj\AppData\Local\Temp\ipykernel_85400\2966875972.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  serie = serie.str.replace('\\n', ' ')


,fname,text
0,legal.txt,iYYU Terms & Conditions These Terms and Condit...
1,privacy.txt,iYYU Privacy Policy This Privacy Policy sets o...
2,training-data.json,"{ ""visibility"": { ""description"": ""By choosi..."


In [15]:
import tiktoken

# Load the cl100k_base tokenizer which is designed to work with the ada-002 model
tokenizer = tiktoken.get_encoding("cl100k_base")

df = pd.read_csv('processed/scraped.csv', index_col=0)
df.columns = ['title', 'text']

# Tokenize the text and save the number of tokens to a new column
df['n_tokens'] = df.text.apply(lambda x: len(tokenizer.encode(x)))

In [16]:
max_tokens = 500


# Function to split the text into chunks of a maximum number of tokens
def split_into_many(text, max_tokens=max_tokens):
    # Split the text into sentences
    sentences = text.split('. ')

    # Get the number of tokens for each sentence
    n_tokens = [len(tokenizer.encode(" " + sentence)) for sentence in sentences]

    chunks = []
    tokens_so_far = 0
    chunk = []

    # Loop through the sentences and tokens joined together in a tuple
    for sentence, token in zip(sentences, n_tokens):

        # If the number of tokens so far plus the number of tokens in the current sentence is greater
        # than the max number of tokens, then add the chunk to the list of chunks and reset
        # the chunk and tokens so far
        if tokens_so_far + token > max_tokens:
            chunks.append(". ".join(chunk) + ".")
            chunk = []
            tokens_so_far = 0

        # If the number of tokens in the current sentence is greater than the max number of
        # tokens, go to the next sentence
        if token > max_tokens:
            continue

        # Otherwise, add the sentence to the chunk and add the number of tokens to the total
        chunk.append(sentence)
        tokens_so_far += token + 1

    return chunks


shortened = []

# Loop through the dataframe
for row in df.iterrows():

    # If the text is None, go to the next row
    if row[1]['text'] is None:
        continue

    # If the number of tokens is greater than the max number of tokens, split the text into chunks
    if row[1]['n_tokens'] > max_tokens:
        shortened += split_into_many(row[1]['text'])

    # Otherwise, add the text to the list of shortened texts
    else:
        shortened.append(row[1]['text'])

In [17]:
df = pd.DataFrame(shortened, columns=['text'])
df['n_tokens'] = df.text.apply(lambda x: len(tokenizer.encode(x)))

In [18]:
import openai
# pip install python-dotenv
from dotenv import load_dotenv
load_dotenv()

openai.api_key = os.getenv('OPENAI_API_KEY')

df['embeddings'] = df.text.apply(lambda x: openai.Embedding.create(input=x, engine='text-embedding-ada-002')['data'][0]['embedding'])
df.to_csv('processed/embeddings.csv')
df.head()

,text,n_tokens,embeddings
0,iYYU Terms & Conditions These Terms and Condit...,480,"[-0.00584253529086709, -0.015280477702617645, ..."
1,By accessing or using (any part of) the Platfo...,478,"[0.015693487599492073, -0.01975192129611969, 0..."
2,You can become a Space Member by agreeing to t...,441,"[0.005822580307722092, -0.028022408485412598, ..."
3,"To the maximum extent permitted by law, iYYU h...",452,"[-0.001226945430971682, -0.013233720324933529,..."
4,"Are not or do not contain spam, are not machin...",492,"[-0.009652386419475079, -0.010911393910646439,..."


In [19]:
df['embeddings'] = df.text.apply(lambda x: openai.Embedding.create(input=x, engine='text-embedding-ada-002')['data'][0]['embedding'])

In [20]:
import pandas as pd
import numpy as np
from openai.embeddings_utils import distances_from_embeddings, cosine_similarity

df = pd.read_csv('processed/embeddings.csv', index_col=0)
df['embeddings'] = df['embeddings'].apply(eval).apply(np.array)

df.head()

,text,n_tokens,embeddings
0,iYYU Terms & Conditions These Terms and Condit...,480,"[-0.00584253529086709, -0.015280477702617645, ..."
1,By accessing or using (any part of) the Platfo...,478,"[0.015693487599492073, -0.01975192129611969, 0..."
2,You can become a Space Member by agreeing to t...,441,"[0.005822580307722092, -0.028022408485412598, ..."
3,"To the maximum extent permitted by law, iYYU h...",452,"[-0.001226945430971682, -0.013233720324933529,..."
4,"Are not or do not contain spam, are not machin...",492,"[-0.009652386419475079, -0.010911393910646439,..."


In [21]:
def create_context(
        question, df, max_len=1800, size="ada"
):
    """
    Create a context for a question by finding the most similar context from the dataframe
    """

    # Get the embeddings for the question
    q_embeddings = openai.Embedding.create(input=question, engine='text-embedding-ada-002')['data'][0]['embedding']

    # Get the distances from the embeddings
    df['distances'] = distances_from_embeddings(q_embeddings, df['embeddings'].values, distance_metric='cosine')

    returns = []
    cur_len = 0

    # Sort by distance and add the text to the context until the context is too long
    for i, row in df.sort_values('distances', ascending=True).iterrows():

        # Add the length of the text to the current length
        cur_len += row['n_tokens'] + 4

        # If the context is too long, break
        if cur_len > max_len:
            break

        # Else add it to the text that is being returned
        returns.append(row["text"])

    # Return the context
    return "\n\n###\n\n".join(returns)


def answer_question(
        df,
        model="gpt-3.5-turbo",
        question="",
        max_len=1800,
        size="ada",
        debug=False,
        max_tokens=150,
        stop_sequence=None
):
    """
    Answer a question based on the most similar context from the dataframe texts
    """
    context = create_context(
        question,
        df,
        max_len=max_len,
        size=size,
    )
    # If debug, print the raw model response
    if debug:
        print("Context:\n" + context)
        print("\n\n")

    try:
        # Create a completions using the question and context
        response = openai.ChatCompletion.create(
            messages=[
                {"role": "system",
                 "content": f"Answer the question based on the context below, and if the question can't be answered based on the context, say \"I don't know. You are the spokesperson of the company iYYU so you will refer to the company by its full name or 'we'. Context: {context}"},
                {"role": "user",
                 "content": question},
            ],
            temperature=0,
            max_tokens=max_tokens,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0,
            stop=stop_sequence,
            model=model,
        )
        return response["choices"][0]['message']['content'].strip()
    except Exception as e:
        print(e)
        return ""

In [22]:
answer_question(df, question="Wat is iYYU? Antwoord in het Nederlands.")

'iYYU is een platform dat gebruikers in staat stelt om in contact te blijven met anderen en berichten te verzenden en ontvangen. Het is eigendom van en wordt beheerd door iYYU B.V., gevestigd in Amsterdam, Nederland.'

In [23]:
answer_question(df, question="Could you give me a small summary of the privacy statement? Explain all the important points as if I were a 5 year old.")

'This is a privacy policy for a website called iYYU. It tells you what information they collect from you when you use their website and how they use it. They promise to keep your information safe and not share it with anyone unless they have to. They also give you some rights, like the right to ask them to delete your information or to see what information they have about you. If you have any questions or concerns, you can contact them.'